# Notebook di training

In [1]:
!pip install -r requirements.txt

In [247]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing as preproc

In [248]:

# LEGGO I FILE DI TEST E DI TRAINING E LI METTO IN DUE DATAFRAME
piazzamenti_training = pd.read_csv('train.csv')
piazzamenti_test = pd.read_csv('test.csv')
piazzamenti_training

,status,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,salary
0,Placed,M,76.76,Others,76.5,Others,Commerce,67.5,Comm&Mgmt,Yes,73.35,Mkt&Fin,64.15,350000.0
1,Placed,M,84.00,Others,90.9,Others,Science,64.5,Sci&Tech,No,86.04,Mkt&Fin,59.42,270000.0
2,Placed,M,65.20,Central,61.4,Central,Commerce,64.8,Comm&Mgmt,Yes,93.40,Mkt&Fin,57.34,270000.0
3,Placed,M,70.00,Central,74.0,Central,Commerce,65.0,Comm&Mgmt,No,83.00,Mkt&Fin,62.28,300000.0
4,Placed,F,61.00,Central,81.0,Central,Commerce,66.4,Comm&Mgmt,No,50.89,Mkt&HR,62.21,278000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,Placed,M,81.00,Others,68.0,Others,Science,64.0,Sci&Tech,No,93.00,Mkt&Fin,62.56,411000.0
125,Placed,M,63.00,Others,67.0,Others,Science,64.0,Sci&Tech,No,75.00,Mkt&Fin,66.46,500000.0
126,Not Placed,M,61.00,Others,70.0,Others,Commerce,64.0,Comm&Mgmt,No,68.50,Mkt&HR,59.50,0.0
127,Placed,M,73.00,Central,78.0,Others,Commerce,65.0,Comm&Mgmt,Yes,95.46,Mkt&Fin,62.16,420000.0


In [249]:
#METTO A 0 IL SALARIO PER STATUS NON PLACED

piazzamenti_training.loc[np.isnan(piazzamenti_training['salary']), 'salary'] = 0.0
piazzamenti_training = piazzamenti_training.dropna()
piazzamenti_training = shuffle(piazzamenti_training)
piazzamenti_training = piazzamenti_training.reset_index(drop=True)


In [250]:
#METTO A 0 IL SALARIO PER STATUS NON PLACED
piazzamenti_test.loc[np.isnan(piazzamenti_test['salary']), 'salary'] = 0.0
piazzamenti_test = piazzamenti_test.dropna()

In [251]:
le = preproc.LabelEncoder()

le.fit(['Placed', 'Not Placed'])

#CODIFICO PLACED CON 1 E NOT PLACED CON 0, LE SALVO IN 2 VARIABILI PER PASSARLE AL METODO FIT E SCORE

output_labels_training = le.transform(piazzamenti_training['status'].values)
output_labels_test = le.transform(piazzamenti_test['status'].values)

#CANCELLO LE COLONNE CHE NON SONO FEATURES

piazzamenti_training = piazzamenti_training.drop(columns='status', axis=1)
piazzamenti_test = piazzamenti_test.drop(columns='status', axis = 1)

In [252]:
def one_hot_encode(dataset):
    return pd.get_dummies(dataset)

In [253]:
# ONE HOT ENCODING TRAINING

ohe = OneHotEncoder(handle_unknown='ignore')
categorical_columns_train = piazzamenti_training.select_dtypes(['category', object]).columns
if 'salary' in categorical_columns_train:
    categorical_columns_train = categorical_columns_train.drop('salary')
dummies_train = pd.DataFrame(ohe.fit_transform(piazzamenti_training[categorical_columns_train]).toarray(), 
                       index=piazzamenti_training.index, 
                       dtype=int)
piazzamenti_training = pd.concat([piazzamenti_training.drop(categorical_columns_train, axis=1), dummies_train], axis=1)


In [254]:
# ONE HOT ENCODING TEST
dummies_test = pd.DataFrame(ohe.transform(piazzamenti_test[categorical_columns_train]).toarray(), 
                       index=piazzamenti_test.index, 
                       dtype=int)
piazzamenti_test = pd.concat([piazzamenti_test.drop(categorical_columns_train, axis=1), dummies_test], axis=1)

In [255]:
# USO UNO SCALER CHE MI PORTA TUTTI I VALORI VICINI TRA DI LORO NEL RANGE [0,1]

scaler = MinMaxScaler()
scaler.fit(piazzamenti_training)
piazzamenti_training = scaler.transform(piazzamenti_training)

scaler.fit(piazzamenti_test)
piazzamenti_test = scaler.transform(piazzamenti_test)

In [256]:
print(np.mean(np.array(piazzamenti_training)))
print(np.var(np.array(piazzamenti_training)))

0.43575439273989436
0.19536549122150385


In [257]:
#ADDESTRO IL MODELLO UTILIZZANDO L'OTTIMIZZAZIONE ADAM, STEP SIZE 1E-5 E UN MASSIMO DI 100000 ITERAZIONI

clf = MLPClassifier(solver='adam', hidden_layer_sizes=(8,), random_state=1, max_iter = 100000)
clf.fit(piazzamenti_training, output_labels_training)

MLPClassifier(hidden_layer_sizes=(8,), max_iter=100000, random_state=1)

In [258]:

print("Accuracy on training set: {:.3f}".format(clf.score(piazzamenti_training, output_labels_training)))
print("Accuracy on test set: {:.3f}".format(clf.score(piazzamenti_test, output_labels_test)))

Accuracy on training set: 0.992
Accuracy on test set: 1.000


In [259]:
cv_scores = cross_val_score(clf, piazzamenti_training, output_labels_training, cv=5)

In [260]:
print(cv_scores)

[0.96153846 0.96153846 0.92307692 0.84615385 0.88      ]


In [261]:
def accuracy(x,y):
    return clf.score(x, y)